<a href="https://colab.research.google.com/github/h-azad/DeepLearning/blob/main/Cat_Dog__Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download tongpython/cat-and-dog

In [ ]:
! unzip cat-and-dog

In [ ]:
import os
import numpy as np
from PIL import Image, ImageTk
from io import BytesIO

import requests
from tkinter import Tk, Label, Canvas, Entry, Button, NW

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.preprocessing import image as image_utils

In [ ]:
model = Sequential()

#Input Layer
model.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

#Hidden Layer
model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

#Output Layer
model.add(Dense(128, activation = "relu"))
model.add(Dense(2, activation = "softmax"))

In [ ]:
model.compile(optimizer = "adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [ ]:
training_datasets = train_datagen.flow_from_directory("training_set/training_set", target_size = (128, 128), batch_size = 32, class_mode = "categorical")
testing_datasets = test_datagen.flow_from_directory("test_set/test_set",  target_size = (128, 128), batch_size = 32, class_mode = "categorical")

In [ ]:
model.fit(training_datasets, steps_per_epoch = 4000/32, epochs = 50, validation_data = testing_datasets, validation_steps = 1000/32)

In [ ]:
! mkdir models

In [ ]:
model.save("models/model.h5")
model.save_weights("models/model_weights.h5")

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 5, figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()

In [ ]:
plotImages([training_datasets[0][0][1] for i in range(0,10)])

In [ ]:
image_width, image_height = 128, 128
model_path = "models/model.h5"
model_wight_path = "models/model_weights.h5"

final_model = load_model(model_path) 
final_model.load_weights(model_wight_path)

In [ ]:
from IPython.display import display as dsp, Image as iImage

In [ ]:
# url = 'https://cdn.pixabay.com/photo/2014/11/30/14/11/cat-551554__340.jpg'
# url = ''

def Enter():
    # global url
    # label.configure()
    # url = (User_input.get())
    # print(url)
    
    url = input("Enter Image Url: ")
    print("=> Input Image Url: ", url)
    
    response = requests.get(url)
    test_image = Image.open(BytesIO(response.content))
    put_image = test_image.resize((400, 400))
    test_image = test_image.resize((128, 128))
    
    
    # img = ImageTk.PhotoImage(put_image)
    # pic = Label(image = img)
    # pic.pack()
    
    # pic.image = img
    test_image = image_utils.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    
    results = model.predict_on_batch(test_image)
    
    
    if results[0][0] == 1:
        res = "Cat"
    elif  results[0][1] == 1:
        res = "Dog"
        
        
    # output = Label(window, text = "Predicted Results: "+ res, font = ("Halvetica", 16))
    # output.pack()
    # print(results[0])
    print("***************************")
    print("** Predicted Result: " +res+" **")
    print("***************************")

    dsp(iImage(response.content))

In [ ]:
Enter()